<a href="https://colab.research.google.com/github/bharathts1507/TSAI-Assignments-EVA4/blob/master/S9_Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from datetime import datetime
print("Current Date/Time: ", datetime.now())

Current Date/Time:  2020-03-17 15:26:27.055729


In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

In [2]:

# mount the drive to access libraries and supportive classes
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.getcwd()
import sys
sys.path.append('/content/drive/My Drive/TSAI_EVA4')

In [0]:
# os.chdir('/content/drive/My Drive/TSAI_EVA4')
# os.listdir()

In [0]:

import DataTransform
channel_means = (0.4914, 0.4822, 0.4465)
channel_stdevs = (0.3025, 0.2918, 0.2679)
trans = DataTransform.Transforms(normalize=True, mean=channel_means, stdev=channel_stdevs)
fillmeans = (np.array(channel_means)*255).astype(np.uint8)
# Train Phase transformations
train_transforms = trans.train_transforms([
                                       #  transforms.Resize((28, 28)),
                                       #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.RandomRotation((-15.0, 15.0), fill=tuple(fillmeans)),
                                       transforms.RandomHorizontalFlip(),
                                       ], 
                                       
                                       [transforms.RandomErasing(value = tuple(channel_means))])

# Test Phase transformations
test_transforms = trans.test_transforms()

In [6]:

#Create Train and Test datasets
from torchvision import datasets
train = datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
test = datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
import DataLoader
dataloader = DataLoader.DataLoader()

# train dataloader
train_loader = dataloader.load(train)

# test dataloader
test_loader = dataloader.load(test)

In [0]:
# from torchsummary import summary
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from trainer import ModelTrainer

# class Net(nn.Module):
#     """
#     Base network that defines helper functions, summary and mapping to device
#     """
#     def conv2d(self, in_channels, out_channels, kernel_size=(3,3), dilation=1, groups=1, padding=1, bias=False, padding_mode="zeros"):
#       return [nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, groups=groups, dilation=dilation, padding=padding, bias=bias, padding_mode=padding_mode)]

#     def separable_conv2d(self, in_channels, out_channels, kernel_size=(3,3), dilation=1, padding=1, bias=False, padding_mode="zeros"):
#       return [nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=kernel_size, groups=in_channels, dilation=dilation, padding=padding, bias=bias, padding_mode=padding_mode),
#               nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(1,1), bias=bias)]

#     def activate(self, l, out_channels, bn=True, dropout=0, relu=True):
#       if bn:
#         l.append(nn.BatchNorm2d(out_channels))
#       if dropout>0:
#         l.append(nn.Dropout(dropout))
#       if relu:
#         l.append(nn.ReLU())

#       return nn.Sequential(*l)

#     def create_conv2d(self, in_channels, out_channels, kernel_size=(3,3), dilation=1, groups=1, padding=1, bias=False, bn=True, dropout=0, relu=True, padding_mode="zeros"):
#       return self.activate(self.conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, groups=groups, dilation=dilation, padding=padding, bias=bias, padding_mode=padding_mode), out_channels, bn, dropout, relu)

#     def create_depthwise_conv2d(self, in_channels, out_channels, kernel_size=(3,3), dilation=1, padding=1, bias=False, bn=True, dropout=0, relu=True, padding_mode="zeros"):
#       return self.activate(self.separable_conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, dilation=dilation, padding=padding, bias=bias, padding_mode=padding_mode),
#                  out_channels, bn, dropout, relu)

#     def __init__(self, name="Model"):
#         super(Net, self).__init__()
#         self.trainer = None
#         self.name = name

#     def summary(self, input_size): #input_size=(1, 28, 28)
#       summary(self, input_size=input_size)

#     def gotrain(self, optimizer, train_loader, test_loader, epochs, statspath, scheduler=None, batch_scheduler=False, L1lambda=0):
#       self.trainer = ModelTrainer(self, optimizer, train_loader, test_loader, statspath, scheduler, batch_scheduler, L1lambda)
#       self.trainer.run(epochs)

#     def stats(self):
#       return self.trainer.stats if self.trainer else None

# class custom_model(Net):
#     def __init__(self, name="Model", dropout_value=0):
#         super(custom_model, self).__init__(name)

#         # Input Convolution: C0
#         # self.conv1 = self.create_conv2d(3, 32, dropout=dropout_value)  # IN 32x32x3, OUT 32x32x32, RF = 3
        
#         self.conv = self.create_conv2d(3, 128 , dropout=dropout_value)  # IN 32x32x3, OUT 32x32x3, RF = 3


#         self.conv1 = self.create_conv2d(128 , 128 , dropout=dropout_value)  # IN 32x32x3, OUT 32x32x3, RF = 3
#         self.conv2 = self.create_conv2d(128 , 128 , dropout=dropout_value) # IN 32x32x32, OUT 32x32x32, RF = 5

#         # Transition 1
#         self.pool1 = nn.MaxPool2d(2, 2) # IN 32x32x32 OUT 16x16x32, RF = 8, jump = 2

        
#         self.conv3 = self.create_conv2d(128 , 128 , dropout=dropout_value) # IN 32x32x32, OUT 32x32x32, RF = 7
#         self.conv4 = self.create_conv2d(128 , 128 , dropout=dropout_value) # IN 16x16x32, OUT 16x16x64, RF = 12
#         self.conv5 = self.create_conv2d(128 , 128 , dropout=dropout_value) # IN 16x16x64, OUT 16x16x64, RF = 16
#         self.pool2 = nn.MaxPool2d(2, 2) # IN 16x16x64 OUT 8x8x64, RF = 18, jump = 4
        


#         # Transition 2
#         # self.dconv1 = self.create_conv2d(64, 128, dilation=2, padding=2) # IN 8x8x64, OUT 8x8x128
#         self.conv6 = self.create_conv2d(128 , 128 , dropout=dropout_value) # IN 8x8x64, OUT 8x8x128, RF = 26
#         self.conv7 = self.create_conv2d(128 , 128 , dropout=dropout_value) # IN 8x8x128, OUT 8x8x128, RF = 34
#         self.conv8 = self.create_conv2d(128 , 128 , dropout=dropout_value) # IN 8x8x128, OUT 8x8x128, RF = 34

       
#         # GAP + FC
#         self.gap = nn.AvgPool2d(kernel_size=(8,8)) 
#         self.conv9 = self.create_conv2d(128, 10, kernel_size=(1,1), padding=0, bn=False, relu=False) # IN: 256 OUT:10


#     def forward(self, x):
#         # x1 =  x    #input = 32x32x3 out=32x32x3
#         x1 = self.conv(x)
#         x2 = self.conv1(x1)  
#         x3 = self.conv2(torch.add(x1, x2)) 
#         x4 = self.pool1(x1 + x2 + x3)
#         # print(x1.shape, x2.shape, x3.shape, "x4", x4.shape)

#         x5 = self.conv3(x4)
#         x6 = self.conv4(x4+x5)
#         x7 = self.conv5(x4+x5+x6)
#         x8 = self.pool2(x5 + x6 + x7)
#         # print(x5.shape, x6.shape, x7.shape, "x8", x8.shape)


#         x9 = self.conv6(x8)
#         x10 = self.conv7(x8 + x9)
#         x11 = self.conv8(x8 + x9 + x10)
        
#         # print(x9.shape, x10.shape, x11.shape)

#         x12 = self.gap(x11)
#         print("x12_gap", x12.shape )
#         x12 = self.conv9(x12)
#         print("x12_conv", x12.shape )
#         x12 = x12.view(-1, 10)

#         return F.log_softmax(x12, dim=-1)


In [0]:
import QuizDNN
model = QuizDNN.custom_model()


In [9]:
# model = custom_model()
use_cuda = torch.cuda.is_available()
model.device = torch.device("cuda" if use_cuda else "cpu")
# print(model.device)
model.to(model.device)
model.summary((3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 32, 32]           3,456
       BatchNorm2d-2          [-1, 128, 32, 32]             256
              ReLU-3          [-1, 128, 32, 32]               0
            Conv2d-4          [-1, 128, 32, 32]         147,456
       BatchNorm2d-5          [-1, 128, 32, 32]             256
              ReLU-6          [-1, 128, 32, 32]               0
            Conv2d-7          [-1, 128, 32, 32]         147,456
       BatchNorm2d-8          [-1, 128, 32, 32]             256
              ReLU-9          [-1, 128, 32, 32]               0
        MaxPool2d-10          [-1, 128, 16, 16]               0
           Conv2d-11          [-1, 128, 16, 16]         147,456
      BatchNorm2d-12          [-1, 128, 16, 16]             256
             ReLU-13          [-1, 128, 16, 16]               0
           Conv2d-14          [-1, 128,

In [11]:
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR
EPOCHS = 30
L2lambda = 0
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True, weight_decay=L2lambda)
#scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.5, steps_per_epoch=len(train_loader), epochs=EPOCHS)
L1lambda = 0
model.gotrain(optimizer, train_loader, test_loader, EPOCHS, "/content/drive/My Drive/TSAI_EVA4", scheduler, True, L1lambda)

Validation loss decreased (100000.000000 --> 1.022379).  Saving model ...
Epoch: 1, Train set: Average loss: 0.0076, Accuracy: 64.88%; Test set: Average loss: 1.0224, Accuracy: 62.88%
Learning Rate = 0.034482


Epoch: 2, Train set: Average loss: 0.0071, Accuracy: 67.79%; Test set: Average loss: 1.2387, Accuracy: 61.06%
Learning Rate = 0.076180


Epoch: 3, Train set: Average loss: 0.0066, Accuracy: 70.05%; Test set: Average loss: 1.0344, Accuracy: 66.73%
Learning Rate = 0.140062


Epoch: 4, Train set: Average loss: 0.0061, Accuracy: 72.78%; Test set: Average loss: 1.2382, Accuracy: 64.81%
Learning Rate = 0.218418


Validation loss decreased (1.022379 --> 0.929462).  Saving model ...
Epoch: 5, Train set: Average loss: 0.0056, Accuracy: 75.21%; Test set: Average loss: 0.9295, Accuracy: 69.92%
Learning Rate = 0.301793


...

Validation loss decreased (0.296682 --> 0.288963).  Saving model ...
Epoch: 26, Train set: Average loss: 0.0016, Accuracy: 92.72%; Test set: Average loss: 0.2890, Accuracy: 91.22%
Learning Rate = 0.043388


Validation loss decreased (0.288963 --> 0.284805).  Saving model ...
Epoch: 27, Train set: Average loss: 0.0015, Accuracy: 93.40%; Test set: Average loss: 0.2848, Accuracy: 91.56%
Learning Rate = 0.024718


Validation loss decreased (0.284805 --> 0.278082).  Saving model ...
Epoch: 28, Train set: Average loss: 0.0014, Accuracy: 93.88%; Test set: Average loss: 0.2781, Accuracy: 91.56%
Learning Rate = 0.011081


Validation loss decreased (0.278082 --> 0.275600).  Saving model ...
Epoch: 29, Train set: Average loss: 0.0013, Accuracy: 94.09%; Test set: Average loss: 0.2756, Accuracy: 91.62%
Learning Rate = 0.002780


Validation loss decreased (0.275600 --> 0.273663).  Saving model ...
Epoch: 30, Train set: Average loss: 0.0012, Accuracy: 94.36%; Test set: Average loss: 0.2737, Accuracy: 91.72%
Learning Rate = 0.000002



In [12]:
from datetime import datetime
print("Current Date/Time: ", datetime.now())

Current Date/Time:  2020-03-17 17:06:21.690488
